In [17]:
'''import sqlite3
from flask import g
DATABASE = '/path/to/database.db'
def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db
@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()'''

#Read Flu_Shot_Locations_-_2014_-_Present.csv from file from current direcory
import csv
import datetime
import pandas
import numpy as np
import sqlite3

In [18]:
fd = open('../data/flushot_locs_19-20.csv', 'r')
reader = csv.reader(fd)
fs_location = []
c = 0
for row in reader:
    if c > 0:
        date = row[15].split("/")
        typed_date = datetime.date(int(date[2]), int(date[0]), int(date[1]))
        #row[15] = arrow.typed_date.format('YYYY-MM-DD')
        row[15] = date[2]+"-"+date[0]+"-"+date[1]
        fs_location.append(row)
    #print(row)
    c += 1
fd.close()

In [19]:

#print(fs_location[0][2:4])
#print(fs_location[1][14:16])


#Define SQL Table
createtbl = """
CREATE TABLE Facility
(
    FacilityID      DECIMAL(3,0),
    Latitude        DOUBLE PRECISION,
    Longtitude      DOUBLE PRECISION,
    Street1         VARCHAR(100),
    PostCode        DECIMAL(6,0),
    FacilityName    VARCHAR(50),
    PhoneNum        DECIMAL(10,0),
    BeginDate       TEXT,
    EndDate         TEXT
);
"""

conn = sqlite3.connect('local.db') # open the connection
cursor = conn.cursor()
cursor.execute(createtbl)   # create the table
conn.commit()   # finalize inserted data

OperationalError: table Facility already exists

In [20]:
for x in range(len(fs_location)):
    if x > 0:
        #DATE in SQLITE date("YYYY-MM-DD")
        #Selecting facilities where flu shot contract is currently active
        cursor.execute("""INSERT OR IGNORE INTO Facility VALUES(?,?,?,?,?,?,?,?,date(?))""", [fs_location[x][1],fs_location[x][2],fs_location[x][3],fs_location[x][4], fs_location[x][8],fs_location[x][10],fs_location[x][12],fs_location[x][14],fs_location[x][15]])
conn.commit()

#Selecting facilities where flu shot contract is currently active
cursor.execute("select Latitude,Longtitude,FacilityID,Street1,PostCode,PhoneNum from Facility where EndDate >= date('now')")
active_facilities = cursor.fetchall()

#Test coordinate will be replaced by current user location coordinate
test_coordinate = [41.178982, -87.625591]

shortest_d = 180 #farthest distance is 180 degrees
# print(active_facilities)
for facility_cordinate in active_facilities:

    distance = ((facility_cordinate[0] - test_coordinate[0])**2 + (facility_cordinate[1] - test_coordinate[1])**2)**(1/2)
    if distance <= shortest_d:
        #Save shortest distance location info
        shortest_d = distance
        closest_index = facility_cordinate[2]
        Street1 = facility_cordinate[3]
        PostCode = facility_cordinate[4]
  
    #distance = np.array(facility_cordinate).sub(np.array(test_coordinate)).pow(2).sum(1).pow(0.5)
    #print(distance)

In [21]:
print("shortest distance is: " + str(shortest_d))
print("closes facility index: " + str(closest_index))
#Query the closet location information
cursor.execute("""select DISTINCT FacilityName,Street1,FacilityID,PostCode from Facility where FacilityID = ? and Street1 = ? and PostCode = ? and EndDate >= date('now')""", [str(closest_index),Street1,str(PostCode)])
facility_name = cursor.fetchall()
print(facility_name)

#close SQLite connection
conn.close()

shortest distance is: 129.50697158379393
closes facility index: 2019-2020
[('United States of America', '-87.6123', '2019-2020', 'IL')]


In [ ]:
def flu_facility_location(address):
    fd = open('../data/flushot_locs_19-20.csv', 'r')
    reader = csv.reader(fd)
    fs_location = []
    c = 0
    for row in reader:
        if c > 0:
            date = row[15].split("/")
            typed_date = datetime.date(int(date[2]), int(date[0]), int(date[1]))
            #row[15] = arrow.typed_date.format('YYYY-MM-DD')
            row[15] = date[2]+"-"+date[0]+"-"+date[1]
            fs_location.append(row)
        #print(row)
        c += 1
    fd.close()


    #print(fs_location[0][2:4])
    #print(fs_location[1][14:16])


    #Define SQL Table
    createtbl = """
    CREATE TABLE Facility
    (
        FacilityID      DECIMAL(3,0),
        Latitude        DOUBLE PRECISION,
        Longtitude      DOUBLE PRECISION,
        Street1         VARCHAR(100),
        PostCode        DECIMAL(6,0),
        FacilityName    VARCHAR(50),
        PhoneNum        DECIMAL(10,0),
        BeginDate       TEXT,
        EndDate         TEXT
    );
    """

    conn = sqlite3.connect('local.db') # open the connection
    cursor = conn.cursor()
    cursor.execute(createtbl)   # create the table
    conn.commit()   # finalize inserted data

    for x in range(len(fs_location)):
        if x > 0:
            #DATE in SQLITE date("YYYY-MM-DD")
            #Selecting facilities where flu shot contract is currently active
            cursor.execute("""INSERT OR IGNORE INTO Facility VALUES(?,?,?,?,?,?,?,?,date(?))""", [fs_location[x][1],fs_location[x][2],fs_location[x][3],fs_location[x][4], fs_location[x][8],fs_location[x][10],fs_location[x][12],fs_location[x][14],fs_location[x][15]])
    conn.commit()

    #Selecting facilities where flu shot contract is currently active
    cursor.execute("select Latitude,Longtitude,FacilityID,Street1,PostCode,PhoneNum from Facility where EndDate >= date('now')")
    active_facilities = cursor.fetchall()

    user_coordinate = convert_to_long_lat(address)

    shortest_d = 180 #farthest distance is 180 degrees
    # print(active_facilities)
    for facility_cordinate in active_facilities:

        distance = ((facility_cordinate[0] - user_coordinate[0])**2 + (facility_cordinate[1] - user_coordinate[1])**2)**(1/2)
        if distance <= shortest_d:
            #Save shortest distance location info
            shortest_d = distance
            closest_index = facility_cordinate[2]
            Street1 = facility_cordinate[3]
            PostCode = facility_cordinate[4]

        #distance = np.array(facility_cordinate).sub(np.array(test_coordinate)).pow(2).sum(1).pow(0.5)
        #print(distance)
    
    return facili

In [1]:
import pandas as pd
from locate import convert_to_long_lat

In [3]:
flu_df = pd.read_csv("../data/flushot_locs_19-20.csv")

In [ ]:
def primary_facility_phone(address):
    """Takes a string address as the user location and returns the closest primary care center location."""

    _latlong = convert_to_long_lat(address)  #<-- This is from the locate.py
    selectedindex = pk_df[["latitude", "longitude"]].sub(_latlong).pow(2).sum(1).pow(0.5).idxmin() 
    
    SELECTED = pk_df.iloc[selectedindex]
    SELECTED_FACILITY = SELECTED[1]
    SELECTED_PHONE = SELECTED[3]
    
    return (SELECTED_FACILITY, SELECTED_PHONE)

In [5]:
flu_df.head()

,Unnamed: 0,Season,Facility ID,Latitude,Longitude,Street1,Street2,City,State,Postal Code,...,Eligibility,Cost,Notes,Record ID,Location,Boundaries - ZIP Codes,Community Areas,Zip Codes,Census Tracts,Wards
0,0,2019-2020,171,41.8967,-87.6241,757 N MICHIGAN AVE,NaN,Chicago,IL,60611,...,NaN,Call for details.,NaN,2019-2020_171,"(41.8967, -87.6241)",6.0,37.0,21182,158.0,11.0
1,1,2019-2020,231,41.8803,-87.6467,111 S HALSTED ST,NaN,Chicago,IL,60661,...,NaN,Call for details.,NaN,2019-2020_231,"(41.8803, -87.6467)",45.0,29.0,4452,786.0,46.0
2,2,2019-2020,108,41.7069,-87.6428,745 WEST 103 ROAD STREET,NaN,Chicago,IL,60628,...,NaN,Call for details. Minimum Age Requirement for ...,NaN,2019-2020_108,"(41.7069, -87.6428)",19.0,45.0,21861,525.0,22.0
3,3,2019-2020,29,41.9192,-87.7313,2121 N. Keeler Ave.,NaN,Chicago,IL,60639,...,NaN,"No cost to individual. If insured, insurance w...",NaN,2019-2020_29,"(41.9192, -87.7313)",2.0,21.0,22615,76.0,12.0
4,4,2019-2020,161,41.9938,-87.6608,6150 N BROADWAY ST,NaN,Chicago,IL,60660,...,NaN,Call for details. Minimum Age Requirement for ...,NaN,2019-2020_161,"(41.9938, -87.6608)",14.0,76.0,4451,536.0,16.0


In [10]:
_latlong = convert_to_long_lat("DePaul Loop Campus")  #<-- This is from the locate.py
selectedindex = flu_df[["Latitude", "Longitude"]].sub(_latlong).pow(2).sum(1).pow(0.5).idxmin() 

SELECTED = flu_df.iloc[selectedindex]
# print(SELECTED)
SELECTED_FACILITY = SELECTED[11]
SELECTED_PHONE = SELECTED[13]
SELECTED_COORDINATE = SELECTED[3],SELECTED[4]

print(SELECTED_COORDINATE[0], SELECTED_COORDINATE[1], SELECTED_FACILITY, SELECTED_PHONE)

41.8798 -87.6273 CVS/pharmacy 312-609-1215


In [11]:
def flu_facility_phone(address):
    """Takes a string address as the user location and returns the closest primary care center location."""
    flu_df = pd.read_csv("../data/flushot_locs_19-20.csv")
    
    _latlong = convert_to_long_lat(address)  #<-- This is from the locate.py
    selectedindex = flu_df[["Latitude", "Longitude"]].sub(_latlong).pow(2).sum(1).pow(0.5).idxmin() 

    SELECTED = flu_df.iloc[selectedindex]
    # print(SELECTED)
    SELECTED_FACILITY = SELECTED[11]
    SELECTED_PHONE = SELECTED[13]
    SELECTED_COORDINATE = SELECTED[3],SELECTED[4]

    return (SELECTED_COORDINATE[0], SELECTED_COORDINATE[1], SELECTED_FACILITY, SELECTED_PHONE)

In [12]:
flu_facility_phone("DePaul Loop Campus")

(41.8798, -87.6273, 'CVS/pharmacy', '312-609-1215')